# Data Processing
Collection of functions for cleaning up raw pressure readings.

## Contents
* [Data Retrieval](#retrieval)
* [Normalization](#normalization)
* [Interpolation](#interpolaton)
* [Masking](#masking)
* [Fourier Transform](#fft)
* [Constrained Inverse Fourier Transform](#ifft)
<hr>

### <a id='retrieval'> Data retrieval </a>

In [8]:
def retrieve_pressure_data(gauge_id,fillNumber,fill=None):   
    fill = db.getLHCFillData(fillNumber) if fill is None else fill
    t1, t2 = fill['startTime'], fill['endTime']
    
    VG = db.get(gauge_id, t1, t2) # Returns a dictionary
    data = pd.DataFrame.from_dict(VG,dtype='float',columns=['Time Stamp','pressure'],orient='index')
    df = pd.DataFrame()
    pressure_readings = data["pressure"][0]
    time_readings = data["Time Stamp"][0]
    
    return pressure_readings,time_readings



In [9]:
def retrieve_context_data(fillNumber, fill = None):
    fill = db.getLHCFillData(fillNumber) if fill is None else fill
    t1, t2 = fill['startTime'], fill['endTime']
    
    energy_variable = 'LHC.BOFSU:OFSU_ENERGY'
    beamEnergy=db.get(energy_variable, t1, t2)
    beam_time, beam_energy = beamEnergy[energy_variable]
    
    return beam_time, beam_energy


In [ ]:
def retrieve_gauge_data(gauge_id,fillNumber,verbose=False, show_plot=False):   
    model,eq_code,location,side,pressure = gauge_id.split(".")
    if verbose:
        print("%s.%s.%s.%s.%s"%(model,eq_code,location,side,pressure))
        
    fill = db.getLHCFillData(fillNumber)
    t1, t2 = fill['startTime'], fill['endTime']
    
    beam_time, beam_energy = retrieve_context_data(fillNumber,fill=fill)
    
    pressure_readings,time_readings = retrieve_pressure_data(gauge_id, fillNumber)

    
    if show_plot:
        f, ax = plt.subplots(1,figsize=(7,5))
        advancedPlottingUtility(axes=[ax],axis_index=0,
                                    xs=[time_readings, beam_time],ys=[pressure_readings, beam_energy],
                                    time_x=True,
                                    plot_labels=(gauge_id,'LHC.BOFSU:OFSU_ENERGY'),
                                    axis_labels=("Time Stamp","Pressure (mbar)", "Energy (GeV)"),
                                    title = "")
    return pressure_readings, time_readings, beam_time, beam_energy


### <a id='normalization'> Normalization </a>

In [10]:
def normalize_y(time_readings, beam_time, pressure_readings, beam_energy, show_plot=False):
    if  np.max(pressure_readings) - np.min(pressure_readings) != 1: #is not normalized already
        norm_pressure_readings = ( pressure_readings - np.min(pressure_readings) ) /  ( np.max(pressure_readings) - np.min(pressure_readings))

    if  np.max(beam_energy) - np.min(beam_energy) != 1: #is not normalized already
        norm_beam_energy = ( beam_energy - np.min(beam_energy) ) /  ( np.max(beam_energy) - np.min(beam_energy))
        
    if show_plot:
        f, ax = plt.subplots(1,figsize=(7,5))
        advancedPlottingUtility(axes=[ax],axis_index=0,
                                xs=[time_readings,beam_time],ys=[norm_pressure_readings,norm_beam_energy],
                                time_x=True,
                                plot_labels=("Vacuum Gauge","Beam Energy"),
                                axis_labels=("Time Stamp","Norm. Pressure","Norm. Energy"),
                                title="Data Normalizaton")
    return norm_pressure_readings,norm_beam_energy


### <a id='interpolaton'> Interpolation </a>

In [11]:
def interpolate_readings(pressure_readings, time_readings, beam_time, beam_energy, density_fact=3, fixed_series_size = None, verbose=False, show_plot = False):
    # Vacuum Gauge
    if fixed_series_size is None:
        t = np.linspace(time_readings[0], time_readings[-1], density_fact*len(time_readings))
    else:
        t = np.linspace(time_readings[0], time_readings[-1], fixed_series_size)
    if verbose:
        print("t {}, time_readings {}, pressure_readings {}".format(t.shape,time_readings.shape,pressure_readings.shape))
        
    pressure_readings = np.interp(t, time_readings, pressure_readings)
    # Beam Intensity
    t2 = np.linspace(beam_time[0], beam_time[-1], density_fact*len(time_readings))
    beam_energy = np.interp(t2, beam_time, beam_energy)

    time_readings = np.linspace(0,1,len(t))
    beam_time = np.linspace(0,1,len(t2))
    
    if show_plot:
        simplePlottingUtility(time_readings,pressure_readings,"Normalized time", "Normalized Pressure",
                          legend_labels="Pressure Readings",
                          title="Interpolation")
    return time_readings,pressure_readings,beam_time,beam_energy



### <a id='masking'> Masking </a>

In [12]:
def double_threshold_energy_masking(time_readings, beam_time,
                       beam_energy,
                       min_distance_frac=0.1,
                       lower_thres = 0.6, upper_thres = 0.9,
                       gradient_thres = 4e-06,
                       verbose = False,
                       show_plot=False):
    gradient = np.gradient(beam_energy)
    min_distance = int(round((min_distance_frac * len(beam_energy))))
    in_critical_region = False
    triggered = False
    cut_off = None
    for i in range(0,len(beam_energy)):
        if in_critical_region:
            if beam_energy[i] < lower_thres:
                in_critical_region = False
            elif beam_energy[i] > upper_thres:
                if verbose:
                    print("TRIGGERED at idx:{},val:{}".format(i,beam_energy[i]))
                in_critical_region = False
                triggered = True
            else:
                pass
        elif beam_energy[i] > lower_thres and beam_energy[i] < upper_thres:
            # print("Entering Critical Region at Index {} with Value {}".format(i,beam_energy[i]))
            in_critical_region = True
        else:
            pass
        
        if triggered:
            # Explore region to right, find maximum by change in gradient
            candidate = None
            j = i
            while candidate is None and j<=len(beam_energy):
                if np.mean(gradient[j:j+10]) <= gradient_thres:
                    candidate = j-1
                j+=1
            if candidate is not None:
                if verbose:
                    print("Candidate at index {}/{} with value {}".format(candidate,len(beam_energy),beam_energy[candidate]))
                if np.all(beam_energy[candidate:candidate+min_distance]>upper_thres):
                    cut_off = beam_time[candidate]
                    mask = time_readings > cut_off
                    if show_plot:
                        simplePlottingUtility(beam_time,beam_energy,"Normalized time", "Normalized Energy",
                                      legend_labels="Beam Energy",
                                      title="Masking",
                                      vertical_line=cut_off)
                        print("Could not find appropriate cut-off")
                    return mask, cut_off
            triggered = False
            

    return None, None


### <a id='fft'> Forward Fourier Transform </a>

In [13]:
def forward_fourier_transform(time_readings,pressure_readings, show_plot=False):
    # Time Spacing (sampling step)
    deltaT= time_readings[2] - time_readings[1]
    # In frequency we have a periodic signal of period fc
    fc=1/deltaT  
    pressure_transform=np.fft.fft(pressure_readings,len(pressure_readings)) #fast fourier transform
    pressure_transform=np.fft.fftshift(pressure_transform)
    pressure_transform=pressure_transform*deltaT
    # Frequency spacing
    deltaF=fc/len(pressure_readings)

    # We are interested only in the range between -fc/2 and fc/2 ( In particular (0,fc/2) beacouse x is real
    f = np.linspace(-fc/2, fc/2 - deltaF, len(pressure_transform)) # Vector of K point from min_value to max_value (Domain in frequency)
    spectrum=[f, np.abs(pressure_transform)]
    
    if show_plot:
        fig,ax = plt.subplots(1,figsize=(7,5))
        advancedPlottingUtility(axes=[ax],axis_index=0,
                        xs=[spectrum[0]],ys=[pressure_transform],
                        plot_labels=("Fourier Transformed Signal"),
                        axis_labels=("Frequency","Amplitude"),
                        title="Forward Fourier Transform")
        
    return pressure_transform, spectrum, deltaT


### <a id='ifft'> Filtered Inverse Fourier Transform </a>

In [14]:
def filtered_inverse_fourier_transform(pressure_transform,deltaT,f,threshold, show_plot = False):
    f_mask = (f < -threshold) | (f > threshold) 
    filtered_frequencies= f[f_mask]
    pressure_transform[f_mask] = 0

    # Constrained inverse fourier transform, wo. higher order terms
    signal_constrained = np.copy(pressure_transform)
    signal_constrained = signal_constrained/deltaT
    signal_constrained = np.fft.ifftshift(signal_constrained)
    signal_constrained = np.fft.ifft(signal_constrained,len(signal_constrained))
    time_constrained = np.linspace(0,1,len(signal_constrained))
    
    if show_plot:
        fig,ax = plt.subplots(1,figsize=(7,5))
        advancedPlottingUtility(axes=[ax],axis_index=0,
                        xs=[time_constrained.real],ys=[signal_constrained.real],
                        plot_labels=("Fourier Transformed Signal"),
                        axis_labels=("Time","Pressure"),
                        title="Noise reduction after filtered inverse FFT")
        
    return time_constrained.real, signal_constrained.real


## <a id='level'> Bin the Data </a>
Split data into segments and return their averages 

In [15]:
def bin_data(data, bins=4):
    segments = np.array_split(data, indices_or_sections=bins)
    if len(data)%bins == 0:
        return np.mean(segments,axis=1)
    else:
        segment_means = np.mean(segments[:len(data)%bins],axis=1)
        remaining_means = np.mean(segments[len(data)%bins:],axis=1)
        segment_means = np.concatenate((segment_means,remaining_means),axis=0)
    return segment_means


## <a id=serializeData> Serialization </a>
Store data in a compressed format

In [16]:
# class processed_gauge_data(object):
 
#     def __init__(self, gauge_id, fill_no, **kwargs): #time_readings, norm_pressure_readings, beam_intensity, beam_time, mask, pressure_levels, gauge_id, fill_no):
#         self.gauge_id = gauge_id
#         self.fill_no = fill_no
#         self.__dict__.update(kwargs)
        
        
#     def generate_data(self,get_levels=True,cuts=8):
#         raw_beam_time,raw_beam_energy = retrieve_context_data(self.fill_no)
#         raw_pressure_readings,raw_time_readings = retrieve_gauge_data(self.gauge_id,self.fill_no)
        
#         if self.raw_pressure_readings.size == 0 or self.raw_time_readings.size == 0:
#             return None
        
#         self.pressure_readings,self.beam_energy = normalize_y(self.raw_time_readings,
#                                                               self.raw_beam_time,
#                                                               self.raw_pressure_readings,
#                                                               self.raw_beam_energy)
        
#         self.time_readings,self.pressure_readings,self.beam_time,self.beam_energy = interpolate_readings(self.pressure_readings,
#                                                                                                         self.raw_time_readings,
#                                                                                                         self.raw_beam_time,
#                                                                                                         self.beam_energy,
#                                                                                                         density_fact=27)
        
#         self.mask,self.threshold = double_threshold_energy_masking(self.time_readings,
#                                                 self.beam_time,
#                                                 self.beam_energy)
#         if self.mask is None or self.threshold is None:
#             raise RuntimeError("Object initialization incomplete, could not load mask")
#         if get_levels:
#             return level_data(self.pressure_readings[self.mask],cuts)
        
#     def __str__(self):
#         string = ""
#         for key, value in self.__dict__.items():
#             if isinstance(value, numpy.ndarray):
#                 string += "%s:\n %s...\n"%(key,value[0:min(4,value.shape[0])])
#             elif isinstance(value,float):
#                 string += "%s:\n %.3f\n"%(key,value)
#             else:
#                 string += "%s:\n %s\n"%(key,value)
#         return string

In [1]:
class processed_gauge_data(object):
 
    def __init__(self, gauge_id, fill_no, **kwargs):
        self.gauge_id = gauge_id
        self.fill_no = fill_no
        self.__dict__.update(kwargs) # Initialize with other information
        
        
    def generate_data(self, bins = None, f_threshold = 40):
        
        if "raw_beam_time" not in self.__dict__ or "raw_beam_energy" not in self.__dict__:
            raw_beam_time,raw_beam_energy = retrieve_context_data(self.fill_no)
        else: 
            raw_beam_time,raw_beam_energy = self.raw_beam_time,self.raw_beam_energy
        if "raw_pressure_readings" not in self.__dict__ or "raw_time_readings" not in self.__dict__:
            raw_pressure_readings,raw_time_readings = retrieve_pressure_data(self.gauge_id,self.fill_no)
        else:
            raw_pressure_readings,raw_time_readings = self.raw_pressure_readings,self.raw_time_readings

        pressure_readings, beam_energy = normalize_y(raw_time_readings, raw_beam_time, raw_pressure_readings, raw_beam_energy)

        time_readings, pressure_readings, beam_time, beam_energy = interpolate_readings(pressure_readings, raw_time_readings, raw_beam_time, beam_energy)

        mask, cut_off = double_threshold_energy_masking(time_readings, beam_time, beam_energy)
        if mask is None:
            raise ValueError("Could not determine mask, data may be too sparse")
        
        pressure_transform, spectrum, deltaT = forward_fourier_transform(time_readings,pressure_readings)

        time_smooth, pressure_smooth = filtered_inverse_fourier_transform(pressure_transform,deltaT,spectrum[0],threshold=f_threshold)

        for attribute in dir(): # Prevents overwritting of any **kwargs provided in Init
            if attribute not in self.__dict__ and attribute != "self":
                setattr(self, attribute, locals()[attribute]) 
        
        if bins is not None:
            return bin_data(self.pressure_readings[self.mask],bins)

    def __str__(self):
        string = ">>>Processed Gauge Data Object<<<\n"
        print(self.__dict__)
        for key, value in self.__dict__.items():
            #print(key,value,"\n")
            if isinstance(value, np.ndarray):
                string += "%s:\n %s...\n"%(key,value[0:min(4,value.shape[0])])
            elif isinstance(value,float):
                string += "%s:\n %.3f\n"%(key,value)
            else:
                string += "%s:\n %s\n"%(key,value)
        return string

In [19]:
# import pytimber
# import pandas as pd
# import numpy as np
# db = pytimber.LoggingDB()
# pgd = processed_gauge_data2("VGPB.418.6L3.B.PR",2216)
# pgd.generate_data()

# print("{}".format(pgd))

t (3183,), time_readings (1061,), pressure_readings (1061,)
{'gauge_id': 'VGPB.418.6L3.B.PR', 'fill_no': 2216, 'beam_energy': array([0.99987183, 0.99670277, 0.98952449, ..., 0.97907458, 0.97919955,
       1.        ]), 'beam_time': array([0.00000000e+00, 3.14267756e-04, 6.28535512e-04, ...,
       9.99371464e-01, 9.99685732e-01, 1.00000000e+00]), 'cut_off': 0.27592708988057824, 'cuts': 8, 'deltaT': 0.00031426775612822125, 'f_threshold': 40, 'get_levels': True, 'mask': array([False, False, False, ...,  True,  True,  True]), 'pressure_readings': array([0.12195122, 0.12941716, 0.13688309, ..., 0.03536573, 0.03231701,
       0.02926829]), 'pressure_smooth': array([0.11855835, 0.11996802, 0.12126715, ..., 0.11369961, 0.11542043,
       0.11704131]), 'pressure_transform': array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j]), 'raw_beam_energy': array([3565.8, 3565.3, 3564.7, ..., 3564.4, 3565.7, 3566.2]), 'raw_beam_time': array([1.31864667e+09, 1.31864667e+09, 1.31864667e+09, ...,
   